In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def normal_pdf(x, mu, sigma):
    """
    Computes the PDF of a normal distribution with mean mu and standard deviation sigma
    at point x.
    """
    return (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-((x - mu) ** 2) / (2 * sigma ** 2))

# Example usage:
x_values = np.linspace(-5, 5, 100)
mu = 0
sigma = 1
pdf_values = normal_pdf(x_values, mu, sigma)

plt.plot(x_values, pdf_values)
plt.title("Normal PDF with mean = 0 and std dev = 1")
plt.xlabel("x")
plt.ylabel("PDF(x)")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Generate some data from a normal distribution
mu = 5
sigma = 2
data = np.random.normal(mu, sigma, 1000)

# Estimate the mean and standard deviation of the distribution from the data
mu_hat, sigma_hat = norm.fit(data)

# Plot a histogram of the data with the estimated distribution overlaid
plt.hist(data, bins=50, density=True, alpha=0.5)
x = np.linspace(mu - 4*sigma, mu + 4*sigma, 100)
plt.plot(x, norm.pdf(x, mu_hat, sigma_hat), 'r-', lw=2)
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# Generate a dataset
data = np.random.normal(5, 2, 1000)

# Define the log-likelihood function
def log_likelihood(theta, x):
    mu, sigma = theta
    return np.sum(norm.logpdf(x, loc=mu, scale=sigma))

# Define the range of mu and sigma values to evaluate the likelihood over
mus = np.linspace(0, 10, 100)
sigmas = np.linspace(0.1, 5, 100)

# Create a grid of mu and sigma values
mu_grid, sigma_grid = np.meshgrid(mus, sigmas)

# Calculate the log-likelihood over the grid of mu and sigma values
ll_grid = np.zeros_like(mu_grid)
for i in range(mu_grid.shape[0]):
    for j in range(mu_grid.shape[1]):
        ll_grid[i,j] = log_likelihood([mu_grid[i,j], sigma_grid[i,j]], data)

# Plot the log-likelihood surface
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(mu_grid, sigma_grid, ll_grid, cmap='viridis')
ax.set_xlabel('mu')
ax.set_ylabel('sigma')
ax.set_zlabel('log likelihood')
plt.show()


In [ ]:
import numpy as np

def kl_divergence(p, q):
    """
    Computes KL divergence between two probability distributions p and q.
    """
    p = np.asarray(p, dtype=np.float)
    q = np.asarray(q, dtype=np.float)
    return np.sum(np.where(p != 0, p * np.log(p / q), 0))

# Example usage:
p = [0.2, 0.3, 0.5]
q = [0.25, 0.25, 0.5]
kl_div = kl_divergence(p, q)
print("KL divergence:", kl_div)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.mixture import GaussianMixture

# Load MNIST dataset
digits = load_digits()
X, y = digits.data, digits.target

# Fit GMM to the data
n_components = 20
gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
gmm.fit(X)

# Generate new samples from the GMM
n_samples = 100
samples, _ = gmm.sample(n_samples)

# Reshape the samples into images
samples = np.reshape(samples, [n_samples, 8, 8])

# Visualize the generated samples
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
for i in range(10):
    for j in range(10):
        axes[i][j].imshow(samples[i * 10 + j], cmap='gray')
        axes[i][j].axis('off')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

# Load MNIST dataset
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape[0], -1)

# Define model
num_components = 100
num_features = x_train.shape[1]
batch_size = 1000
gmm = tfp.distributions.MixtureSameFamily(
    mixture_distribution=tfp.distributions.Categorical(
        probs=tf.ones([num_components], dtype=tf.float32) / num_components),
    components_distribution=tfp.distributions.MultivariateNormalDiag(
        loc=tf.Variable(tf.zeros([num_components, num_features], dtype=tf.float32)),
        scale_diag=tfp.util.TransformedVariable(tf.ones([num_components, num_features], dtype=tf.float32),
                                                bijector=tfb.Softplus())
    )
)

# Define loss function
@tf.function
def nll_loss(x):
    return -tf.reduce_mean(gmm.log_prob(x))

# Define optimizer
optimizer = tf.optimizers.Adam(learning_rate=0.001)

# Train model
num_epochs = 100
num_batches = x_train.shape[0] // batch_size

for epoch in range(num_epochs):
    epoch_loss = 0.
    for batch in range(num_batches):
        x_batch = x_train[batch*batch_size:(batch+1)*batch_size]
        with tf.GradientTape() as tape:
            loss = nll_loss(x_batch)
        gradients = tape.gradient(loss, gmm.trainable_variables)
        optimizer.apply_gradients(zip(gradients, gmm.trainable_variables))
        epoch_loss += loss
    print('Epoch: {}, Loss: {}'.format(epoch, epoch_loss / num_batches))

# Generate samples from the trained model
num_samples = 10
samples = gmm.sample(num_samples)
samples = tf.reshape(samples, [num_samples, 28, 28])
samples = tf.clip_by_value(samples, 0, 1)
